In [1]:
from openai import OpenAI
import json

In [5]:
import os
bool(os.environ["OPENAI_API_KEY"])

True

In [2]:
client = OpenAI()

In [3]:
def get_weather(location: str, unit: str = "celsius"):
    """Mock weather lookup"""
    data = {
        "chennai": 33,
        "delhi": 29,
        "mumbai": 30,
    }
    return {
        "location": location,
        "temperature": data.get(location.lower(), 25),
        "unit": unit,
    }

def get_time(city: str):
    """Mock time lookup"""
    times = {
        "chennai": "2025-09-24 18:30",
        "delhi": "2025-09-24 18:00",
        "london": "2025-09-24 14:00",
    }
    return {"city": city, "time": times.get(city.lower(), "2025-09-24 12:00")}

def calculator(expression: str):
    """Simple math calculator"""
    return {"expression": expression, "result": eval(expression)}



In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "Get the current time in a given city",
            "parameters": {
                "type": "object",
                "properties": {"city": {"type": "string"}},
                "required": ["city"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "Calculate the result of a mathematical expression",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {"type": "string"},
                },
                "required": ["expression"],
            },
        },
    },
]

# Tool registry
tool_registry = {
    "get_weather": get_weather,
    "get_time": get_time,
    "calculator": calculator,
}

In [5]:
messages = [
    {"role": "user", "content": "whats 2+2*9/8*6*7.1*7/22/33*8"}
]

response = client.chat.completions.create(
    model="gpt-4.1-nano",   # can switch to "gpt-4.1-nano" if enabled
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

message = response.choices[0].message
print("Model output:", message)

Model output: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_h9W7woB3FaHrZPMYUFmaM9oq', function=Function(arguments='{"expression":"2+2*9/8*6*7.1*7/22/33*8"}', name='calculator'), type='function')])


In [18]:


# Handle tool calls dynamically
if message.tool_calls:
    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        params = json.loads(tool_call.function.arguments)

        func = tool_registry.get(tool_name)
        if func:
            result = func(**params)
        else:
            result = {"error": f"Unknown tool: {tool_name}"}

        # Send back to model
        messages.append(message)
        messages.append(
            {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
        )

        final_response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=messages,
        )
        print("Final Answer:", final_response.choices[0].message.content)
else:
    print("LLM reply:", message.content)

Final Answer: The result of the expression \(2 + 2 \times 9 / 8 \times 6 \times 7.1 \times 7 / 22 / 33 \times 8\) is approximately 9.39.
